# Implementación 
El siguiente código muestra un sistema basado en el procesamiento de imágenes para detectar, extraer y clasificar la lesión a partir de imágenes dermatoscopicas (imágenes que proceden de un dermatoscopio), esto con el fin de ayudar en el diagnóstico de cáncer de piel melanoma. 

Primero importamos las imágenes de lesiones cutáneas benignas y malignas, cada imagen esta acompañada por un conjunto de datos relativos al paciente y a la lesión como la edad, sexo, lugar de la lesión, el diagnostico (si es benigna o maligna) y el tipo de estructura. Este conjunto de datos provienen del Memorial Sloan Kettering Cancer Center.

Segundo, creamos el conjunto data, el cual contiene imágenes de lesiones clasificadas como benignas, y el conjunto data2 contiene imágenes de lesiones cutáneas clasificadas como malignas. Despues seleccionamos la imagen que queremos analizar y a que tipo de lesión corresponde.

OpenCV es una biblioteca empleada para el análisis y tratamiento de imágenes, uno de los usos más importantes de esta librería en la visión por computadora es la detección de rostros y objetos. Para analizar y caracterizar las lesiones cutáneas, es necesario realizar distintas técnicas a la imagen, como segmentación, por eso es utilizamos esta librería.

Es necesario elegir una sección de la imagen original para que sea posible apreciar la lesión de forma mas detallada, cuando hacemos este paso simplemente estamos delimitando la región de interés (ROI),
Una vez ampliada la imagen, convertimos la imagen a escala de grises pues para segmentar usamos el método thresholding, el cual se realiza en imagenes en escala de grises. Como estamos interesados en identificar características de la piel se implementa el filtrado de ruido mediante el filtro mediano para reducir el impacto objetos fuera de nuestro interés, como el cabello.

Thresholding es una técnica de segmentación usada para separar un objeto considerado como primer plano de su fondo. Usamos el método de thresholding de Otsu ya que el ROI es homogéneo y, por lo tanto, el thresholding se vuelve dinámico en función del histograma de la imagen mejorada. (“The Melanoma Skin Cancer Detection and Classification Using Support Vector Machine” - October 2017 - DOI:10.1109/AEECT.2017.8257738). Después de eso, se aplica el relleno de imagen para eliminar los pixeles del fondo del interior del objeto detectado y, haciendo así el ROI claro.

Una vez diferenciadas el área de la lesión de su fondo, seleccionamos el objeto que nos interesa, es decir, la región de la lesión cutánea a estudiar. El "enmascaramiento" (mask) de imágenes implica resaltar un objeto específico dentro de una imagen enmascarándola.

Una vez segmentada la imagen, procederemos a analizar distintas características en la lesión, como color, simetria, diametro, y bordes irregulares. Usamos Image opening para eliminar el exceso de pixels de fondo, y tambien para suavizar el controno que delimita el objeto y elimina protuberancias delgadas. Después de que la región de la imagen recortada y convertida en escala de grises, realizamos ecualización del histograma a la imagen de nivel de gris extraída.

Después de extraer la lesión (ROI) en la etapa de segmentación, las características predefinidas se extraerán del ROI para clasificación. Las características seleccionadas son forma, color y
varias características de textura. Las
características dermatoscópicas (ABCD) son importantes para distinguir los tipos de lesiones cutáneas. Combinamos estas características para
obtener buenos resultados de clasificación y distinción de  las lesiones benignas de las lesiones cutáneas malignas. Dado que la señal de advertencia más importante del melanoma es un cambio en el tamaño, la forma o el color de un lunar o de otro crecimiento en la piel, es importante prestar especial atención a la asimetría. 

Para la extracción de las características podemos pensar el código como tres partes: 

*Para la parte (A) usamos la imagen en el formato original RGB ( contiene tres canales de colores, rojo, verde y azul) para así calcular la densidad de colores específicos en el imagen de la lesión.

*Para la parte (B) tratamos con el binario, donde las características de asimetría, irregularidad de los bordes y circulación se obtienen de la imagen binaria. Estas características se calculan con parámetros como Asimetría, Irregularidad de borde, color y diámetro. 

*Para la fase (C) usamos la imagen de lesión en imagen en escala de grises. Aquí hallamos los valores de energía, correlación y homogeneidad.

In [115]:
#Librerías
from PIL import Image, ImageEnhance, ImageOps
import requests
import cv2 as cv
import numpy as np
#Para imprimir en colab
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from skimage.feature import greycomatrix, greycoprops
from scipy.stats import kurtosis
import json
from sklearn.model_selection import train_test_split
import cvxopt

In [116]:
#Benignos
response_API = requests.get("https://api.isic-archive.com/api/v2/images/search/?limit=140&query=benign_malignant%3Abenign")
data=json.loads(response_API.text)

#Malignos
response_API2 = requests.get("https://api.isic-archive.com/api/v2/images/search/?limit=140&query=benign_malignant%3Amalignant")
data2=json.loads(response_API2.text)

In [117]:

#Extración de muestra de imagenes benignas:

benignList = list(range(0,140))
benRemList = [3,21,25,30,39,40,41,61,66,71,75,80,82,93,95,96,98,101,103,104,108,112,113,120,121,124,125,130,133,135]

for i in benRemList:
    benignList.remove(i)

In [118]:

#Extración de muestra de imagenes malignas:

malignList = list(range(0,140))
malRemList = [0,2,6,12,22,25,31,33,40,43,49,55,60,65,66,69,77,90,91,100,103,104,113,118,121,122,123,125,130,133,134,138,139]

for i in malRemList:
    malignList.remove(i)

In [119]:

#Extracción de las características de las imagenes 

def Features(Listtst):
    stdGreenM=[]
    stdBlueM=[]
    stdRedM=[]
    energyM=[]
    contrastM=[]
    homogeneityM=[]
    dissimilarityM=[]
    correlationM=[]
    compM=[]
    IrrIdxAM=[]
    CirIdM=[]
    mediaEqM=[]
    kurtEqM=[]
    typeMb=[]
    kernel = np.ones((5, 5), 'uint8')
    for i in Listtst:
        rll=data2["results"][i]["files"]["full"]["url"]
        #Abrimos imagen
        im = Image.open(requests.get(rll, stream=True).raw)
        
        opencvImage = cv.cvtColor(np.array(im), cv.COLOR_RGB2BGR)
        ROI = opencvImage[800:2000, 1000:2400]

        #Convertimos a blanco y negro
        grayImg = cv.cvtColor(ROI, cv.COLOR_BGR2GRAY)
        #Quitamos ruido de la imagen
        noiseImg = cv.fastNlMeansDenoising(grayImg)

        #Otsu threshholding
        ret, thresh = cv.threshold(noiseImg,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
        
        #Se realiza dilatación de la imagen
        dilImg = cv.dilate(thresh, kernel, iterations=1)

        #Para sacar lo segmentado
        thresh = cv.bitwise_not(dilImg)
        contours, hierarchy = cv.findContours(thresh,cv.RETR_TREE,cv.CHAIN_APPROX_NONE)

        cnt = max(contours, key=cv.contourArea)

        h, w = ROI.shape[:2]
        mask = np.zeros((h, w), np.uint8)

        cv.drawContours(mask, [cnt],-1, 255, -1)
        rest = cv.bitwise_and(ROI, ROI, mask=mask)

        #Para sacar la desviacion estandar de colores (creo que hay que normalizar)

        histTestBl = cv.calcHist([rest],[0],mask,[256],[0,256])
        bl = np.std(histTestBl)
        histTestGr = cv.calcHist([rest],[1],mask,[256],[0,256])
        gr = np.std(histTestGr)
        histTestRe = cv.calcHist([rest],[2],mask,[256],[0,256])
        re = np.std(histTestRe)

        color = [bl,gr,re]
        maxCol = max(color)
        stdGreenM.append(gr/maxCol)
        stdBlueM.append(bl/maxCol)
        stdRedM.append(re/maxCol)

        restNoise = cv.bitwise_and(noiseImg, noiseImg, mask=mask)

        #Otras características dada la matriz de co-ocurrencia de nivel gris
        GLMC=greycomatrix(restNoise, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256)
        energy=greycoprops(GLMC,'energy')[0,0]
        contrast=greycoprops(GLMC,'contrast')[0,0]
        homogeneity=greycoprops(GLMC,'homogeneity')[0,0]
        dissimilarity=greycoprops(GLMC,'dissimilarity')[0,0]
        correlation=greycoprops(GLMC,'correlation')[0,0]

        energyM.append(energy)
        contrastM.append(contrast)
        homogeneityM.append(homogeneity)
        dissimilarityM.append(dissimilarity)
        correlationM.append(correlation)

        #Para mirar simetría (parece que funciona bien xdddxdxdxd)

        gray = cv.cvtColor(ROI, cv.COLOR_BGR2GRAY)
        blur = cv.GaussianBlur(gray, (5, 5), 0)
        ret,thresh = cv.threshold(blur,70,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
        contours, hierarchy = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        max_cnt = max(contours, key=cv.contourArea)

        ellipse = cv.fitEllipse(max_cnt)
        ellipse_pnts = cv.ellipse2Poly( (int(ellipse[0][0]),int(ellipse[0][1]) ) ,( int(ellipse[1][0]),int(ellipse[1][1]) ),int(ellipse[2]),0,360,1)
        comp = cv.matchShapes(max_cnt,ellipse_pnts,1,0.0)
        compM.append(comp)

        #Intentando sacar irregularidad / diametro:
        #Area de lesión
        ImgArea= cv.countNonZero(restNoise)
        #Dimensiones del ROI
        TotalArea = 1200*1400
        #numero de pixeles en contorno
        perimetro = len(max_cnt)
        #Irregularity index A
        IrrA = perimetro/ImgArea
        IrrIdxAM.append(IrrA)
        #Circularity index
        CirId = (4*np.pi*ImgArea)/perimetro**2
        CirIdM.append(CirId)

        #Equalizacion de histograma
        equImg = cv.equalizeHist(restNoise)
        equHist = histTest = cv.calcHist([rest],[0],mask,[256],[0,256])
        mediaEq = np.mean(equHist)
        mediaEqM.append(mediaEq)
        kurtEq = kurtosis(equHist)
        kurtEqM.append(kurtEq[0])



    return stdGreenM,stdBlueM,stdRedM,energyM,contrastM,homogeneityM, dissimilarityM, correlationM, compM, IrrIdxAM, CirIdM,mediaEqM, kurtEqM

In [120]:
#Extraccion de características para las imagenes benignas       benigno(label) -> -1

stdGreenB=[]
stdBlueB=[]
stdRedB=[]
energyB=[]
contrastB=[]
homogeneityB=[]
dissimilarityB=[]
correlationB=[]
compB = []
IrrIdxAB=[]
CirIdB=[]
mediaEqB=[]
kurtEqB=[]
typeBb=[-1]*len(benignList)

stdGreenB,stdBlueB,stdRedB,energyB,contrastB,homogeneityB,dissimilarityB,correlationB,compB,IrrIdxAB,CirIdB,mediaEqB,kurtEqB = Features(benignList)

In [121]:
#Extraccion de características para las imagenes malignas      malignas(label) -> -1

stdGreenMal=[]          
stdBlueMal=[]           
stdRedMal=[]           
energyMal=[]
contrastMal=[]
homogeneityMal=[]
dissimilarityMal=[]
correlationMal=[]
compMal=[]              
IrrIdxAMal=[]           
CirIdMal=[]             
mediaEqMal=[]
kurtEqMal=[]
typeMalb=[1]*len(malignList)

stdGreenMal,stdBlueMal,stdRedMal,energyMal,contrastMal,homogeneityMal,dissimilarityMal,correlationMal,compMal,IrrIdxAMal,CirIdMal,mediaEqMal,kurtEqMal = Features(malignList)


In [122]:
#Construcción de los vectores de características
featVec_Mal = []
for i in range(0,len(malignList)):
    SingVec = [stdGreenMal[i], stdBlueMal[i], stdRedMal[i], energyMal[i], contrastMal[i], homogeneityMal[i], dissimilarityMal[i], correlationMal[i],compMal[i],IrrIdxAMal[i],CirIdMal[i], mediaEqMal[i], kurtEqMal[i]]
    featVec_Mal.append(SingVec)

#Indice si es maligno
typeMalb=np.array([1]*len(malignList))
print(featVec_Mal)

featVec_Bien = []
for i in range(0,len(benignList)):
    SingVec = [stdGreenB[i], stdBlueB[i], stdRedB[i], energyB[i], contrastB[i], homogeneityB[i], dissimilarityB[i], correlationB[i],compB[i],IrrIdxAB[i],CirIdB[i], mediaEqB[i], kurtEqB[i]]
    featVec_Bien.append(SingVec)

#Indice si es benigno
typeBb=np.array([-1]*len(benignList))
print(featVec_Bien)

[[0.8054746, 1.0, 0.75715244, 0.8755320493025326, 24.725658803907557, 0.9521509142047961, 0.304151775077436, 0.9903933127156653, 0.5344186111460459, 0.017718427934163974, 0.1924107562811441, 812.625, 2.5237875], [0.85758716, 0.71701014, 1.0, 0.9055223220159295, 27.147280200142955, 0.976587011768915, 0.21129378127233736, 0.9936298375139757, 0.06573692462611497, 0.011125322931968998, 0.6428734297332227, 616.90625, 2.3220615], [0.88512814, 0.69484323, 1.0, 0.9563970852822379, 14.853173695496777, 0.9835719901390811, 0.12796878722897306, 0.9922387972887999, 0.035231909489878144, 0.015027059696162193, 0.764396219253579, 284.3828, -0.7636306], [0.89184636, 0.87053484, 1.0, 0.7446194080682997, 43.99844412675719, 0.9184960245397618, 0.4575792232547059, 0.9939778871218248, 0.06589975033281131, 0.006335594821177252, 0.7319024113679309, 1670.8672, -0.6395657], [0.9191918, 0.93523836, 1.0, 0.9327999078621991, 12.69096616630928, 0.9745263675246318, 0.17105908982606632, 0.9904807671313273, 0.04471856

In [123]:
#Pasamos a numpy array

featVec_Mal = np.array(featVec_Mal)
featVec_Bien = np.array(featVec_Bien)

In [124]:
#Junta todos los vectores de características de todas las imagenes

X = np.vstack((featVec_Mal,featVec_Bien))
print(np.shape(X))
Y = np.concatenate((typeMalb,typeBb))

print(np.shape(X))
print(np.shape(Y))

#Escoje la muestra de entrenamiento y de prueba
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.65, random_state=42)

(217, 13)
(217, 13)
(217,)


In [125]:

#Support vector machines

def gaussian(x, z, sigma=0.1):
    return np.exp(-np.linalg.norm(x - z, axis=1) ** 2 / (2 * (sigma ** 2)))


class SVM:
    def __init__(self, kernel=gaussian, C=1):
        self.kernel = kernel
        self.C = C
    def fit(self, X, y):
        self.y = y
        self.X = X
        m, n = X.shape

        # Calculate Kernel
        self.K = np.zeros((m, m))
        for i in range(m):
            self.K[i, :] = self.kernel(X[i, np.newaxis], self.X)

        # Solve with cvxopt final QP needs to be reformulated
        # to match the input form for cvxopt.solvers.qp
        P = cvxopt.matrix(np.outer(y, y) * self.K)
        q = cvxopt.matrix(-np.ones((m, 1)))
        G = cvxopt.matrix(np.vstack((np.eye(m) * -1, np.eye(m))))
        h = cvxopt.matrix(np.hstack((np.zeros(m), np.ones(m) * self.C)))
        A = cvxopt.matrix(y, (1, m), "d")
        b = cvxopt.matrix(np.zeros(1))
        cvxopt.solvers.options["show_progress"] = False
        sol = cvxopt.solvers.qp(P, q, G, h, A, b)
        self.alphas = np.array(sol["x"])

    def predict(self, X):
        y_predict = np.zeros((X.shape[0]))
        sv = self.get_parameters(self.alphas)

        for i in range(X.shape[0]):
            y_predict[i] = np.sum(
                self.alphas[sv]
                * self.y[sv, np.newaxis]
                * self.kernel(X[i], self.X[sv])[:, np.newaxis]
            )

        return np.sign(y_predict + self.b)

    def get_parameters(self, alphas):
        threshold = 1e-5

        sv = ((alphas > threshold) * (alphas < self.C)).flatten()
        self.w = np.dot(self.X[sv].T, alphas[sv] * self.y[sv, np.newaxis])
        self.b = np.mean(
            self.y[sv, np.newaxis]
            - self.alphas[sv] * self.y[sv, np.newaxis] * self.K[sv, sv][:, np.newaxis]
        )
        return sv


if __name__ == "__main__":

    np.random.seed(1)
    svm = SVM(kernel=gaussian)
    svm.fit(X_train, y_train)
    y_predxd = svm.predict(X_train)

    #Presición de entrenamiento
    print(f"Presición de entrenamiento: {sum(y_train==y_predxd)/y_train.shape[0]}")

Presición de entrenamiento: 0.8133333333333334


In [126]:
y_pred = svm.predict(X_test)
#Presición de test
print(f"Presición de test: {sum(y_test==y_pred)/y_test.shape[0]}")

Presición de test: 0.3873239436619718


**(Navegacion entre los notebooks)**

- Ir a [presentación del problema](Presentacion_Problema.ipynb)

- Ir a [modelo](Modelo.ipynb)

---

**Autores:** Alejandro Martin Salcedo, David Alexander Núñez Quintero, Paula Ximena Rodriguez Nempeque.

---